Sweeper on all qubit drives at same time

Single shot, calibrate qubit states Averaging single shot ?

Bandwidth +-800 Mhz a bit less

Exactly, for multiplexed readout you use software modulation, in which case you are then also limited in the readout and integration length by the available waveform memory. For the spectroscopy mode with hardware modulation, these limitation are much relaxed

Integration mode is the standard readout mode. it differs from spectroscopy mode in that you can multiplex multiple readout pulses and demodulate the signal also at multiple frequencies. However, the readout pulse length and integration kernel length is limited to 2us per channel.
In spectroscopy mode you can use the hardware oscillation for modulation of the signal and demodulation of the return, but you only have a single frequency available. If you play pulses in spectroscopy mode, you are using the so called pulsed spectroscopy mode, where the total pulse length is limited to 16 or 32 us, depending on instrument options. But still a single frequency / pulse

# Qubit Spectroscopy

In [9]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import Zurich

import numpy as np
from qibolab.pulses import Pulse, PulseSequence, ReadoutPulse, PulseShape
from qibocal.data import DataUnits
import laboneq.simple as lo

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = Zurich('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

2023.01.23 12:52:32.294 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is /home/admin/Juan/qibolab/examples/laboneq_output/log
2023.01.23 12:52:32.296 laboneq.controller.controller  INFO   VERSION: laboneq 1.6.0
2023.01.23 12:52:32.296 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.01.23 12:52:32.476 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.01.23 12:52:33.840 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.01.23 12:52:34.065 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [ ]:
sequence = PulseSequence() 
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_sequences([sequence])
print(msr, phase, i , q)

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def frequency_msr_phase__fast_precision(data):
        
    data_fast = data

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("MSR", "uV"),
            name="Fast",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("phase", "rad"),
            name="Fast",
        ),
        row=1,
        col=2,
    )
    
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Frequency (GHz)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Frequency (GHz)",
        yaxis2_title="Phase (rad)",
    )
    return fig

In [11]:
Device.reload_settings()
Device.apply_settings()

In [12]:
from qibolab.sweeper import Sweeper

def qubit_spectroscopy_sweeper(
    Device,
    qubits,
    start,
    stop,
    around,
    points,
):
    
    sequence = PulseSequence() 
    sweepers = []
    
    for qubit in qubits:
        
        around = Device.instruments["shfqc_qc"]["settings"][f"if_frequency_{qubit}"]
        
        ro_pulse = Device.create_qubit_readout_pulse(qubit, start=0)
        sequence.add(ro_pulse)
        qd_pulse = Device.create_qubit_drive_pulse(qubit, start=0, duration=1000)
        sequence.add(qd_pulse)
    
        sweepers.append(Sweeper(f"freq_qs_{qubit}", start, stop, points,around, [qd_pulse]))
    
    return sequence, sweepers

In [ ]:
modulation_type=lo.ModulationType.HARDWARE,
# modulation_type=lo.ModulationType.SOFTWARE

acquisition_type=lo.AcquisitionType.INTEGRATION,
averaging_mode=lo.AveragingMode.CYCLIC,

In [13]:
start = -100e6
stop = 100e6
points = 50

qubits = [0,1,2,3,4]
# qubits = [0]

Base_if = Device.instruments["shfqc_qc"]["settings"][f"if_frequency_{0}"]
    
sequence, sweepers = qubit_spectroscopy_sweeper(
    Device,
    qubits = qubits,
    start = start,
    stop = stop,
    around = Base_if,
    points = points,
)
Base_lo = Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]
    
msr, phase, i, q = Device.execute_sequences([sequence], sweepers)

for qubit in qubits:

    Data = DataUnits(
        name=f"sweep_q{qubit}", quantities={"frequency": "Hz"}
    )
    
    results = {
        "MSR[V]": msr[qubit],
        "i[V]": i[qubit],
        "q[V]": q[qubit],
        "phase[rad]": phase[qubit],
        "frequency[Hz]": sweepers[qubit].values + Base_lo,
    }
    
    Data.set(results)

    frequency_msr_phase__fast_precision(Data).show()


2023.01.23 12:52:40.681 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.23 12:52:41.391 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 164
2023.01.23 12:52:41.391 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 40000
2023.01.23 12:52:41.392 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.23 12:52:43.457 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.23 12:52:44.721 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



In [9]:
lo.show_pulse_sheet("Pulse_sheet",Device.exp)

2023.01.20 17:27:33.771 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to /home/admin/Juan/qibolab/examples/Pulse_sheet_2023-01-20-17-27-33.html


/home/admin/Juan/qibolab/examples/Pulse_sheet_2023-01-20-17-27-33.html

Check QS

In [28]:
Device.reload_settings()
# Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = freq
Device.apply_settings()

In [29]:
sequence = PulseSequence()
qd_pulse = Device.create_qubit_drive_pulse(0, start=0, duration=1000) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=1000)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_sequences([sequence])
print(msr, phase, i , q)

2022.12.09 13:10:06.449 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 13:10:06.475 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 40
2022.12.09 13:10:06.476 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2022.12.09 13:10:06.476 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2022.12.09 13:10:07.209 laboneq.controller.controller  INFO   Starting near-time execution...
2022.12.09 13:10:08.236 laboneq.controller.controller  INFO   Finished near-time execution.
4.457683753667133e-05 -2.230818304052491 -2.733158423175067e-05 -3.521475469392777e-05
2022.12.09 13:10:08.584 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 13:10:08.609 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 39
2022.12.09 13:10:08.609 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2022.12.09 13:10:08.609 laboneq